In [14]:
class GeneticOptimizatorJSSP:
    # CONSTRUCTOR
    def __init__(self):
        self.jobs = []
        self.solutions = []

    # GET/SET JOBS
    def get_jobs(self):
        return self.jobs
    def set_jobs_from_string(self, jobs):
        _jobs = []
        for line in jobs.strip().split('\n'):
            tokens = line.split()
            _job = []
            for idx in range(0, len(tokens), 2):
                _job.append([tokens[idx], tokens[idx+1]])
            _jobs.append(_job)
        self.jobs = _jobs
    def set_jobs_from_file(self, jobs):
        _jobs = []
        with open(jobs, 'r')  as file:
            for line in file:
                tokens = line.split()
                _job = []
                for idx in range(0, len(tokens), 2):
                    _job.append([tokens[idx], tokens[idx+1]])
                _jobs.append(_job)
        self.jobs = _jobs

    # GENETIC ALGORITHM
    def _initialize_population(self, population_size):
        # TODO: Implement population initialization logic
        pass
    def _evaluate_fitness(self, population):
        # TODO: Implement fitness evaluation logic
        pass
    def _selection(self, population):
        # TODO: Implement selection process
        pass
    def _crossover(self, inidividual_1, individual_2):
        # TODO: Implement crossover process
        pass
    def _mutate(self, individual):
        # TODO: Implement mutation process
        pass
    def _elitism(self, popuplation_1, population_2):
        # TODO: Implement elitism process
        pass

    def run_genetic_algorithm(self, num_generations, population_size):
        # Initialize population P
        _P = _initialize_population(population_size)
        # Evaluate fitness of all individuals in P
        _evaluate_fitness(_P, self.problem)
        # Iterate over generations
        for generation in range(num_generations):
            _P_prime = []
            # Iterate over pairs of individuals
            for _ in range(population_size//2):
                # Selection
                _c_a, _c_b = _selection(_P)
                # Crossover
                _c_prime_a, _c_prime_b = _crossover(_c_a, _c_b)
                # Mutation
                _mutate(_c_prime_a)
                _mutate(_c_prime_b)
                # Add to the new population
                _P_prime.append(_c_prime_a)
                _P_prime.append(_c_prime_b)
            # Elitism
            _P_prime = _elitism(_P, _P_prime)
            # Update population
            _P = _P_prime
            # Evaluate fitness of all individuals in P
            _evaluate_fitness(_P, self.problem)
        self.solutions = _P

In [15]:
# Create and instance of the genetic optimizator
optimizator = GeneticOptimizatorJSSP()

# Define the problem from a file
optimizator.set_jobs_from_file('problem.txt')

# Define the problme from a string
# problem = """
# 0 88 1 68
# 1 72 0 50
# """
# optimizator.set_jobs_from_string(problem)

# Print the problem to check that was imported properly
print(optimizator.get_jobs())


[[['0', '88'], ['1', '68']], [['1', '72'], ['0', '50']]]
